In [1]:

!pip install gradio

import gradio as gr

# Rule-Based Chatbot

FAQ_REPLIES = {
    "hello": "Hello there! How can I help you today?",
    "hi": "Hi! How can I assist you?",
    "what is your name": "I’m a simple rule-based chatbot demo!",
    "how do i reset my password": "To reset your password, go to the login screen and click 'Forgot Password'.",
    "bye": "Goodbye! Have a great day."
}

def rule_based_chatbot(user_input):

    user_input_lower = user_input.strip().lower()

    # Check if the user input is in the dictionary
    if user_input_lower in FAQ_REPLIES:
        return FAQ_REPLIES[user_input_lower]
    else:
        return "I’m not sure how to handle that. Can you rephrase?"

# Create a Gradio interface
demo_rule_based = gr.Interface(
    fn=rule_based_chatbot,
    inputs="text",
    outputs="text",
    title="Rule-Based Chatbot Demo",
    description="A minimal rule-based chatbot that looks up fixed replies."
)

demo_rule_based.launch()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 2.3 MB/s eta 0:00:00
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://484342a9d62357e604.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spa

In [2]:

!pip install transformers gradio

import gradio as gr
from transformers import pipeline


# Zero-Shot Classifier

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Define the possible "intents" (labels) our bot can handle:
POSSIBLE_LABELS = ["greeting", "faq", "complaint", "goodbye"]

def model_driven_chatbot(user_input):

    # Zero-shot classification
    classification_result = classifier(user_input, POSSIBLE_LABELS)

    # classification_result["labels"] is sorted by predicted likelihood
    predicted_label = classification_result["labels"][0]

    # Rule-based response after we identify the label
    if predicted_label == "greeting":
        return "Hello! How can I help you?"
    elif predicted_label == "faq":
        return "Let me see if I can answer that question..."
    elif predicted_label == "complaint":
        return "I'm sorry to hear that. Could you tell me more about the issue?"
    elif predicted_label == "goodbye":
        return "Goodbye! Have a wonderful day."
    else:
        return "I'm not entirely sure how to handle that, but I'm learning!"

# Gradio interface
demo_model_driven = gr.Interface(
    fn=model_driven_chatbot,
    inputs="text",
    outputs="text",
    title="Model-Driven Chatbot Demo (Zero-Shot)",
    description=(
        "Try entering a greeting, FAQ question, complaint, or goodbye. "
        "The bot uses a transformer model to identify your intent."
    )
)


demo_model_driven.launch()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0eb7224ae0da84a7fe.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


###
PIZZA EXAMPLE
###

In [3]:
!pip install gradio

import gradio as gr
from transformers import pipeline

# Initialize the zero-shot classifier (using facebook/bart-large-mnli)
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Candidate labels now include an explicit "out_of_scope" option.
CANDIDATE_LABELS = ["greeting", "order_pizza", "menu_question", "out_of_scope"]

# Known pizza types for simple entity extraction simulation.
PIZZA_TYPES = ["pepperoni", "margherita", "vegetarian", "cheese"]

# Confidence threshold for deciding whether a prediction is reliable.
THRESHOLD = 0.7

def conversation_flow(user_input: str, state: dict):

    if state is None:
        state = {"stage": "initial", "order": None}

    text = user_input.strip().lower()

    # Check for immediate goodbye commands.
    if "bye" in text or "goodbye" in text:
        response = "Goodbye! Have a great day. (Resetting conversation.)"
        state = {"stage": "initial", "order": None}
        return response, state

    # Use zero-shot classification to evaluate the input.
    result = classifier(text, CANDIDATE_LABELS)
    top_label = result["labels"][0]
    top_score = result["scores"][0]

    # If confidence is too low or out_of_scope is predicted, return a fallback message.
    if top_score < THRESHOLD or top_label == "out_of_scope":
        response = "I'm not sure I understand. Could you please clarify your request?"
        return response, state

    # Stage: initial
    if state["stage"] == "initial":
        if top_label == "greeting":
            response = "Hello! How can I help you today? You can say something like 'I want to order a pizza.'"
            state["stage"] = "awaiting_intent"
        elif top_label == "order_pizza":
            response = "Great! What type of pizza would you like? (e.g., pepperoni, margherita)"
            state["stage"] = "awaiting_pizza_type"
        elif top_label == "menu_question":
            response = "Our menu includes pepperoni, margherita, vegetarian, and cheese pizzas. Would you like to place an order?"
            state["stage"] = "awaiting_intent"
        else:
            response = "I'm not sure I understand. Could you please clarify your request?"
            state["stage"] = "awaiting_intent"
        return response, state

    # Stage: awaiting_intent
    elif state["stage"] == "awaiting_intent":
        if top_label == "order_pizza":
            response = "Great! What type of pizza would you like? (e.g., pepperoni, margherita, vegetarian)"
            state["stage"] = "awaiting_pizza_type"
        elif top_label == "menu_question":
            response = "Our menu includes pepperoni, margherita, vegetarian, and cheese pizzas. Would you like to place an order?"
        elif top_label == "greeting":
            response = "Hi again! How can I assist you? If you'd like to order a pizza, just let me know."
        else:
            response = "Could you please specify your request? For example, say 'I want to order a pizza.'"
        return response, state

    # Stage: awaiting_pizza_type
    elif state["stage"] == "awaiting_pizza_type":
        found_type = None
        for pizza in PIZZA_TYPES:
            if pizza in text:
                found_type = pizza
                break
        if found_type:
            response = f"Got it! Your order for a {found_type} pizza has been placed. Thank you!"
            state["order"] = found_type
            state["stage"] = "order_confirmed"
        else:
            response = "I didn't catch the pizza type. Could you please specify (e.g., pepperoni, margherita, vegetarian, cheese)?"
        return response, state

    # Stage: order_confirmed
    elif state["stage"] == "order_confirmed":
        response = "Your order is confirmed. If you'd like to order another pizza or have more questions, just let me know!"
        # Reset state for new orders.
        state["stage"] = "awaiting_intent"
        state["order"] = None
        return response, state

    # Fallback: reset conversation.
    else:
        response = "I'm not sure how to proceed. Let's start over. How can I help you today?"
        state = {"stage": "initial", "order": None}
        return response, state

def chat_interface(user_message, history, state):

    if state is None:
        state = {"stage": "initial", "order": None}
    response, state = conversation_flow(user_message, state)
    history = history or []
    history.append((user_message, response))
    return history, state


with gr.Blocks() as demo_chat:
    gr.Markdown("# Pizza Ordering Chatbot")
    gr.Markdown(
        "This chatbot helps you order pizza"

    )
    chatbot = gr.Chatbot()
    state = gr.State({"stage": "initial", "order": None})
    txt = gr.Textbox(show_label=False, placeholder="Type your message here and press Enter")
    txt.submit(chat_interface, inputs=[txt, chatbot, state], outputs=[chatbot, state])


demo_chat.launch()



Device set to use cpu
<ipython-input-3-296ab15a44b7>:115: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://190e6c42dca54d4a6e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
